In [1]:
# EJERCICIO 1 - APIs

# para hacer solicitudes HTTP
import requests

# para manipulación de datos
import pandas as pd

In [10]:
# cada una pone la kpi que haya generado

api_key = "12b5dc93-7799-439e-ac66-3b0eda416a3f"

# 12b5dc93-7799-439e-ac66-3b0eda416a3f - este es el apikey que sacó Nerea

# Primero hacemos 

jugadores = {"nombres":[], "apellidos":[], "equipo":[]}

# Estos son los parámetros de la solicitud. Vamos a pedir los 200 jugadores, para que nos muestre 100 por página:
parametros = {'page': 200,'per_page': 100}

res_jugadores = requests.get(f"http://api.balldontlie.io/v1/players", params = parametros, headers = {'Authorization': api_key} ) 

# llamada de los jugadores:
# el json() nos sirve para estructurar los datos que intercambian el servidor y la llamada

res_jugadores = res_jugadores.json()


In [11]:
# Explicacion del FOR: por cada página en el rango primera página a la tercera, metemos otro for dentro, para recorrer todos los jugadores:

for page in range(1, 3):
    for jugador in res_jugadores["data"]:
        jugadores["nombres"].append(jugador["first_name"])
        jugadores["apellidos"].append(jugador["last_name"])
        jugadores["equipo"].append(jugador["team"])

# estructuracion de datos

dataframe_jugadores = pd.DataFrame(jugadores)
dataframe_jugadores

,nombres,apellidos,equipo
0,Alex,Abrines,"{'id': 21, 'conference': 'West', 'division': '..."
1,Jaylen,Adams,"{'id': 1, 'conference': 'East', 'division': 'S..."
2,Steven,Adams,"{'id': 11, 'conference': 'West', 'division': '..."
3,Bam,Adebayo,"{'id': 16, 'conference': 'East', 'division': '..."
4,DeVaughn,Akoon-Purcell,"{'id': 8, 'conference': 'West', 'division': 'N..."
...,...,...,...
195,Joe,Chealey,"{'id': 4, 'conference': 'East', 'division': 'S..."
196,Marquese,Chriss,"{'id': 7, 'conference': 'West', 'division': 'S..."
197,Gary,Clark,"{'id': 19, 'conference': 'West', 'division': '..."
198,Ian,Clark,"{'id': 10, 'conference': 'West', 'division': '..."


In [26]:
partidos_request = requests.get(f"https://api.balldontlie.io/v1/games?per_page=100?page={pagina}" )
partidos_request

<Response [401]>

In [23]:
# Extraer la información de 200 partidos:

partidos = {"equipo_local":[], "equipo_visitante":[],"puntos_local":[], "puntos_visitante": [],"temporada":[]}

# Hacemos el request dentro del for porque se hace un request por cada página.

for pagina in range (1, 3):

    try:
        partidos_request = requests.get(f"https://api.balldontlie.io/api/v1/games?per_page=100?page={pagina}" ) 
        partidos_request = partidos_request.json()

        for partido in partidos_request["data"]:
            partidos["equipo_local"].append(partido["home_team"]["full_name"])
            partidos["equipo_visitante"].append(partido["visitor_team"]["full_name"])
            partidos["puntos_local"].append(partido["home_team_score"])
            partidos["puntos_visitante"].append(partido["visitor_team_score"])   
            partidos["temporada"].append(partido["season"]) 

    except requests.exceptions.RequestException as e:
        print(f"Error al realizar la solicitud en la página {pagina}: {e}")
    
    except ValueError as e:

        print(f"Error al convertir la respuesta a JSON en la página {pagina}: {e}")


partidos_dataframe = pd.DataFrame(partidos)                          
partidos_dataframe


Error al realizar la solicitud en la página 1: Expecting value: line 1 column 1 (char 0)
Error al realizar la solicitud en la página 2: Expecting value: line 1 column 1 (char 0)


,equipo_local,equipo_visitante,puntos_local,puntos_visitante,temporada


In [27]:
# ESTE ES EL DE LOS EJERCICIOS RESUELTOS:

partidos = {"equipo_local": [], "equipo_visitante": [], "puntos_local": [], "puntos_visitante": [], "temporada": []}

for pagina in range(1, 3):
        
    res_partidos = requests.get(f"https://www.balldontlie.io/api/v1/games?per_page=100?page={pagina}")
    res_partidos = res_partidos.json()

    
    for partido in res_partidos["data"]:
        partidos["equipo_local"].append(partido["home_team"]["full_name"])
        partidos["equipo_visitante"].append(partido["visitor_team"]["full_name"])
        partidos["puntos_local"].append(partido["home_team_score"])  
        partidos["puntos_visitante"].append(partido["visitor_team_score"])   
        partidos["temporada"].append(partido["season"]) 

df_partidos = pd.DataFrame(partidos)
df_partidos

JSONDecodeError: Expecting value: line 2 column 1 (char 1)

In [16]:
# Buscar el nombre, apellidos, y equipo de los 200 jugadores de la NBA

jugadores = llamada.json()

jugadores.keys()

dict_keys(['data', 'meta'])

In [ ]:
print

In [27]:
# fist_name, last_name
# ojo team es otro diccionario donde tiene datos del equipo: team:name

diccionario_jugadores = {"first_name":[]}
# diccionario_jugadores = {"first_name":[],"last_name":[],"team":[]}

for jugador in jugadores:
    diccionario_jugadores["first_name"].append(data[first_name])


print(diccionario_jugadores[data])





NameError: name 'data' is not defined

In [ ]:
def mostrar_informacion(url):
    global informacion_jugadores
    informacion_jugadores = []
    for jugador in jugadores:
        

## PARTE 2 WEB SCRAPPING
